### Redes con tensorflow
#### Por Francisco Serradilla

#### Tareas

* Entrenar el perceptrón multicapa suministrado.
* Ampliarlo para meter más capas, usar activación relu y evaluar con un conjunto de test.
* Entrenar con algunos de los problemas suministrados.
* (Opcional) Probar otros optimizadores.
* (Opcional) Añadir dropout. ¿Mejora la generalización con alguno de los problemas suministrados?

In [1]:
# Creación de grafos

print('Loading tensorflow...')
import tensorflow as tf
print('Loaded')

# Create a Constant op that produces a 1x2 matrix.  The op is
# added as a node to the default graph.
# The value returned by the constructor represents the output of the Constant op.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.],[2.]])

# Create a Matmul op that takes 'matrix1' and 'matrix2' as inputs.
# The returned value, 'product', represents the result of the matrix multiplication.
product = tf.matmul(matrix1, matrix2)

product

Loading tensorflow...
Loaded


<tf.Tensor 'MatMul:0' shape=(1, 1) dtype=float32>

In [2]:
# Ejecutar un grafo

# Launch the default graph.
sess = tf.Session()
# To run the matmul op we call the session 'run()' method, passing ‘product' which represents the output of the matmul op.  This indicates to the call that we want to get the output of the matmul op back.
# All inputs needed by the op are run automatically by the session.  They typically are run in parallel.
# The call 'run(product)' thus causes the execution of three ops in the graph: the two constants and matmul. The output of the op is returned in 'result' as a numpy `ndarray` object.
result = sess.run(product)
print(result)
# ==> [[ 12.]]

# Close the Session when we're done.
sess.close()

[[12.]]


In [3]:
# Variables

# Create a Variable, that will be initialized to the scalar value 0.
state = tf.Variable(0, name="counter")

# Create an Op to add one to `state`.
one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)

# Variables must be initialized by running an `init` Op after having launched the graph.  
init_op = tf.global_variables_initializer() # We first have to add the `init` Op to the graph

with tf.Session() as sess: # Launch the graph and run the ops
  sess.run(init_op) # Run the 'init' op
  print(sess.run(state)) # Print the initial value of 'state'
  for _ in range(3): # Run the op that updates 'state' and print 'state'.
    sess.run(update)
    print(sess.run(state))

Instructions for updating:
Colocations handled automatically by placer.
0
1
2
3


In [4]:
# fetches

input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed]) # fetches -> lo que se quiere obtener del cálculo
  print(result)

[21.0, 7.0]


In [5]:
# placeholders

input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
  print(sess.run(output, feed_dict={input1:[7.], input2:[2.]}))

[14.]


In [30]:
# Carga ejemplos

import numpy as np

ni = 2 # número de entradas
nh = 3 # número de neuronas en capa oculta
no = 3 # número de neuronas en capa de salida
capas = 4 #número de capas de la red

# carga datos de entrenamiento
d = np.loadtxt("samples/circulo.txt")
inputs = d[:,:ni]

outputs = d[:,ni:]


In [31]:
# Creación de la red
# Define un perceptrón multicapa con 2 capas usando RMS como medida del error y back proagation como algoritmo de entrenamiento

import tensorflow as tf

# un placeholder es un punto de comunicación de datos entre nuestra app y la librería tensorflow
# creamos un placeholder para indicar las entradas a la red de todos los ejemplos
e = tf.placeholder(tf.float32, [None, ni]) # None indica que la primera dimensión no es fija (el número de ejemplos)

print("e")
print(e)
# un nuevo placeholder para indicar las salidas deseadas
d = tf.placeholder(tf.float32, [None, no])
print("d")
print(d)
# las variables son matrices que residen fuera de python y no son modificadas directamente por nuestra app
W = []
b = []

W.append(tf.Variable(tf.glorot_uniform_initializer()((ni, nh),dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))
for _ in range (1,capas-1):
    W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, nh), dtype=tf.float32)))
    b.append(tf.Variable(tf.glorot_uniform_initializer()([nh], dtype=tf.float32)))

W.append(tf.Variable(tf.glorot_uniform_initializer()((nh, no),  dtype=tf.float32)))
b.append(tf.Variable(tf.glorot_uniform_initializer()([no], dtype=tf.float32)))

# aquí definimos la propagación de la red
s = []
s.append(tf.nn.relu(tf.add(tf.matmul(e, W[0]), b[0])))
for i in range (capas-1):
    s.append(tf.nn.relu(tf.add(tf.matmul(s[i], W[i+1]), b[i+1])))

e
Tensor("Placeholder_20:0", shape=(?, 2), dtype=float32)
d
Tensor("Placeholder_21:0", shape=(?, 3), dtype=float32)


In [32]:
# Entrenamiento

# definimos la función a minimizar (error cuadrático medio)
print("d")
print(d)
print("\ns")
print(s)
vRMS = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(d,s)), reduction_indices=0))
RMS = tf.reduce_mean(vRMS)

# definimos el algoritmo de aprendizaje a utilizar (descenso del gradiente sobre la función de coste)
train_step = tf.train.AdamOptimizer().minimize(RMS)

# definimos qué es un acierto  
correct_prediction = tf.equal(tf.argmax(s,1), tf.argmax(d,1))

# definimos la precisión como el porcentaje de aciertos
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# definimos la inicialización de las variables internas
init = tf.global_variables_initializer()

# iniciamos sesión con tensorflow y ejecutamos la inicialización
sess = tf.Session()
sess.run(init)

# ejecutamos el número de epochs indicados
epochs = 1000
trace = 100
for i in range(1,epochs+1):
  sess.run(train_step, feed_dict={e: inputs, d: outputs})
  if i%trace == 0:
    print ('Epoch: %d' % i)
    # calculamos RMS y precisión y la escribimos por pantalla
    print('RMS:',sess.run(RMS, feed_dict={e: inputs, d: outputs}))
    print ('Accuracy:',sess.run(accuracy, feed_dict={e: inputs, d: outputs}))
    
sess.close()

d
Tensor("Placeholder_21:0", shape=(?, 3), dtype=float32)

s
[<tf.Tensor 'Relu_32:0' shape=(?, 3) dtype=float32>, <tf.Tensor 'Relu_33:0' shape=(?, 3) dtype=float32>, <tf.Tensor 'Relu_34:0' shape=(?, 3) dtype=float32>, <tf.Tensor 'Relu_35:0' shape=(?, 3) dtype=float32>]
Epoch: 100
RMS: nan


InvalidArgumentError: Incompatible shapes: [4,3] vs. [25]
	 [[node Equal_1 (defined at <ipython-input-32-6b494188da14>:15) ]]

Caused by op 'Equal_1', defined at:
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/Cellar/ipython/7.2.0/li